<a href="https://colab.research.google.com/github/timbosssds/ML/blob/main/Chatbot_with_citations_wip_convomemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install required libraries
#!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers
#!pip install pypdf
#!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.7 MB/s eta 0:00:00


In [1]:
# MileStone 1
# -- end-to-end (No Citations)

# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
list_items = texts
text = convert_list_to_string_with_speech_marks(list_items)

# -- Model
context=text
from transformers import pipeline
def generate_answer(question, context):
  """Generates an answer to a question given a context."""
  question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
  response = question_answerer(question=question, context=text, max_answer_len=1024, top_p=1)
  response = response[list(response)[-1]]
  return response

# -- Call model and produce answer
# Generate an answer to the question
#question = "what is the Eligibility criteria for the Product"
question = "what is the date"

answer = generate_answer(question, context)
print(answer)

1 October 2022


In [2]:
# MileStone 2
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
list_items = texts
text = convert_list_to_string_with_speech_marks(list_items)

# -- Model
context=text
# Define a function to generate an answer to a question with citations
from transformers import pipeline

def generate_answer_with_citations(question, context):
    """Generates an answer to a question given a context and provides citations."""
    question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
    response = question_answerer(question=question, context=context, max_answer_len=1024, top_p=1)
    answer_text = response[list(response)[-1]]
    answer_start = response["start"]
    answer_end = response["end"]

    # Get the passage containing the answer
    start_index = max(0, context.rfind(".", 0, answer_start))
    end_index = min(len(context), context.find(".", answer_end))
    passage_with_answer = context[start_index:end_index].strip()

    # Get the citation (page number) of the passage
    page_number = context.count(".", 0, start_index) + 1

    return {
        "answer": answer_text,
        "passage": passage_with_answer,
        "citation": f"Page {page_number}"
    }

# Example usage
question = "what is the date"
answer_info = generate_answer_with_citations(question, context)
print("Answer:", answer_info["answer"])
print("Passage:", answer_info["passage"])
print("Citation:", answer_info["citation"])

Answer: 1 October 2022
Passage: .
Dated 1 October 2022
We are required under the Corporations Act to 
provide you with the following  information:
This Financial Services Guide (FSG) is issued by 
Commonwealth Bank of Australia (‘Bank’, ‘we’, 
‘our’ or ‘us’) and is intended to provide you with 
information that will assist you in making informed 
decisions on whether you want to use our products 
and services
Citation: Page 1


In [ ]:
# -- All below is WIP...

In [24]:
# WIP - trying to get conversation history
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
  # Get the page object
  page = loader.pages[page_num]
  # Extract the texst from the page and add it to the text variable
  text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
  list_items = texts
  text = convert_list_to_string_with_speech_marks(list_items)

# -- 4.0 Maintain conversation history
conversation_history = []

# -- Model
context=text

# Define a function to generate an answer to a question with citations
from transformers import pipeline

def generate_answer_with_citations(question, context):
  """Generates an answer to a question given a context and provides citations."""
  # Check if the question is related to any of the previous questions
  for previous_question, previous_answer in conversation_history:
    if question.startswith(previous_question):
      context = f"{context} {previous_answer}"

  question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
  response = question_answerer(question=question, context=context, max_answer_len=1024, top_p=1)
  answer_text = response[list(response)[-1]]
  answer_start = response["start"]
  answer_end = response["end"]

  # Get the passage containing the answer
  start_index = max(0, context.rfind(".", 0, answer_start))
  end_index = min(len(context), context.find(".", answer_end))
  passage_with_answer = context[start_index:end_index].strip()

  # Get the citation (page number) of the passage
  page_number = context.count(".", 0, start_index) + 1

  # Add the new question and answer to the conversation history
  conversation_history.append((question, answer_text))

  return {
      "answer": answer_text,
      "passage": passage_with_answer,
      "citation": f"Page {page_number}"}

# Example usage
#question = "what is the date" # original question

#question = "what is your GPO box number?" # initial q to test follow up question
question = "what state is that in?" # follow up question to see if conversation memory working
answer_info = generate_answer_with_citations(question, context)
print("Answer:", answer_info["answer"])
print("Passage:", answer_info["passage"])
print("Citation:", answer_info["citation"])


Answer: Colonial First State
Passage: .
2  Avanteos Investments Limited is a subsidiary of Superannuation and 
Investments HoldCo Pty Limited ABN 64 644 660 882 (HoldCo) and is 
part of Colonial First State, which refers to HoldCo and its subsidiaries
Citation: Page 10


In [23]:
conversation_history

[('what is your GPO box number?', '2719')]